# Curso de Word2Vec: treinamento de Word Embedding

In [1]:
import pandas as pd

# 1. Explorando o Spacy

In [2]:
df_treino = pd.read_csv('/home/franciscofoz/Documents/GitHub/machine-learning-training/Processamento de linguagem natural/Datasets/w2v3_treino.csv')
df_teste = pd.read_csv('/home/franciscofoz/Documents/GitHub/machine-learning-training/Processamento de linguagem natural/Datasets/w2v3_teste.csv')

In [3]:
df_treino

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...
...,...,...,...,...,...,...
89995,"Mural: Há 30 anos, aeroporto não foi bem receb...",Década de 1970. Congonhas já estava superlotad...,2015-01-22,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/01...
89996,"As notícias sobre Schumacher não são boas, diz...",O ex-presidente da Ferrari Luca di Montezemolo...,2016-04-02,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/02/1...
89997,"De olho em R$ 50 bilhões, governo pode concede...","Para fazer caixa, o governo estuda conceder pa...",2017-08-29,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...
89998,Moro deu a Lula o papel de coitadinho,Realizou-se parcialmente o primeiro objetivo d...,2016-06-03,colunas,eliogaspari,http://www1.folha.uol.com.br/colunas/eliogaspa...


In [4]:
import spacy

nlp = spacy.load('pt_core_news_sm')

2024-04-26 20:06:29.816245: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
texto = 'Rio de Janeiro é uma cidade maravilhosa'
doc = nlp(texto)

In [8]:
type(doc)

spacy.tokens.doc.Doc

In [9]:
doc[0]

Rio

In [10]:
type(doc[0])

spacy.tokens.token.Token

In [11]:
doc.ents

(Rio de Janeiro,)

In [12]:
type(doc.ents)

tuple

In [13]:
doc[1]

de

In [14]:
doc[1].is_stop

True

# 2. Pré processamento com Spacy

In [18]:
texto_para_tratamento = (titulos.lower() for titulos in df_treino.title)

In [23]:
def trata_textos(doc):
    
    tokens_validos = list()
    
    for token in doc:
        eh_valido = not token.is_stop and token.is_alpha
        if eh_valido:
            tokens_validos.append(token.text)
            
    if len(tokens_validos) > 2:
        return ' '.join(tokens_validos)
    

In [25]:
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [27]:
from time import time

t0 = time()

textos_tratados = [trata_textos(doc) for doc in nlp.pipe(texto_para_tratamentos,batch_size=1000,n_process= -1)]

tf = time() - t0

print(tf/60)

1.732930342356364


In [28]:
titulos_tratados = pd.DataFrame({'titulos':textos_tratados})

In [29]:
titulos_tratados

,titulos
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc
...,...
89995,mural há anos aeroporto recebido moradores gua...
89996,notícias schumacher boas ferrari
89997,olho bilhões governo conceder áreas petróleo
89998,moro deu lula papel coitadinho


# 3. Hiperparâmetros do Word2Vec

In [34]:
from gensim.models import Word2Vec


w2v_modelo = Word2Vec(sg=0,
                      window=2,
                      vector_size=300,
                      min_count=5,
                      alpha=0.03,
                      min_alpha=0.007)

In [38]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

90000
84466


In [41]:
lista_lista_tokens = [titulo.split(' ') for titulo in titulos_tratados.titulos]

In [43]:
import logging

logging.basicConfig(format='%(asctime)s : - %(message)s',level = logging.INFO)

w2v_modelo.build_vocab(lista_lista_tokens,progress_per=5000)




2024-04-26 21:16:43,943 : - collecting all words and their counts
2024-04-26 21:16:43,944 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-04-26 21:16:43,963 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2024-04-26 21:16:43,991 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2024-04-26 21:16:44,011 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2024-04-26 21:16:44,026 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2024-04-26 21:16:44,059 : - PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word types
2024-04-26 21:16:44,084 : - PROGRESS: at sentence #30000, processed 191554 words, keeping 25494 word types
2024-04-26 21:16:44,116 : - PROGRESS: at sentence #35000, processed 223412 words, keeping 27330 word types
2024-04-26 21:16:44,147 : - PROGRESS: at sentence #40000, processed 255282 words, keeping 29053

# 4. Treinamento do Word2Vec

In [44]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30)

2024-04-26 21:19:35,053 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2024-04-26T21:19:35.053758', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-6.5.0-28-generic-x86_64-with-glibc2.35', 'event': 'train'}
2024-04-26 21:19:36,100 : - EPOCH 0 - PROGRESS: at 88.85% examples, 420424 words/s, in_qsize 5, out_qsize 0
2024-04-26 21:19:36,226 : - EPOCH 0: training on 540242 raw words (486174 effective words) took 1.2s, 421452 effective words/s
2024-04-26 21:19:37,257 : - EPOCH 1 - PROGRESS: at 96.22% examples, 464359 words/s, in_qsize 3, out_qsize 0
2024-04-26 21:19:37,264 : - EPOCH 1: training on 540242 raw words (486200 effective words) took 1.0s, 479086 effective words/s
2024-04-26 21:19:38,097 : - EPOCH 2: training on 540242 raw words (486185 effective words) took 0.8s, 597026 effective words/s

(14584390, 16207260)

In [47]:
w2v_modelo.wv.most_similar('revista')

[('aliança', 0.36559873819351196),
 ('hugh', 0.3606097996234894),
 ('criador', 0.35564297437667847),
 ('playboy', 0.34625309705734253),
 ('cosac', 0.34071919322013855),
 ('dublador', 0.33473655581474304),
 ('allen', 0.3341161608695984),
 ('filósofa', 0.3315010964870453),
 ('law', 0.33079516887664795),
 ('cards', 0.32965970039367676)]

In [52]:
w2v_modelo.wv.most_similar('livro')

[('romance', 0.4368336796760559),
 ('biografia', 0.4339773654937744),
 ('livros', 0.407418817281723),
 ('virginia', 0.40444430708885193),
 ('memórias', 0.40106022357940674),
 ('autobiografia', 0.3997996747493744),
 ('automóvel', 0.3987919092178345),
 ('coletânea', 0.39591890573501587),
 ('antologia', 0.38114482164382935),
 ('álbuns', 0.37838220596313477)]

In [55]:
w2v_modelo.wv.most_similar('coca')

[('cultivo', 0.5499607920646667),
 ('firmam', 0.4404166638851166),
 ('culinária', 0.43562230467796326),
 ('homicídios', 0.4202877879142761),
 ('publicados', 0.4139198660850525),
 ('hidrelétricas', 0.4076575040817261),
 ('gastronomia', 0.4067721366882324),
 ('acumulado', 0.40170738101005554),
 ('açúcar', 0.4001285433769226),
 ('assinantes', 0.3992401361465454)]

In [56]:
w2v_modelo.wv.most_similar('gato')

[('viúva', 0.4742732346057892),
 ('orson', 0.4596574008464813),
 ('agressor', 0.4594123363494873),
 ('valter', 0.441806823015213),
 ('monólogo', 0.43999508023262024),
 ('banal', 0.435576468706131),
 ('cauã', 0.435467392206192),
 ('lou', 0.43429383635520935),
 ('eisenstein', 0.4333285093307495),
 ('dela', 0.43278568983078003)]

# 4. Treinamento do modelo